In [163]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import binarize, LabelEncoder, MinMaxScaler
from sklearn import metrics
from sklearn.metrics import accuracy_score, mean_squared_error, precision_recall_curve
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

df = pd.read_csv('https://raw.githubusercontent.com/z5208980/machine-learning-health/main/symptom_predictor/data/data.csv')
df_severity = pd.read_csv('https://raw.githubusercontent.com/z5208980/machine-learning-health/main/symptom_predictor/data/severity.csv')
 
df.head()

,Disease,Symptom_1,Symptom_2,Symptom_3,Symptom_4,Symptom_5,Symptom_6,Symptom_7,Symptom_8,Symptom_9,Symptom_10,Symptom_11,Symptom_12,Symptom_13,Symptom_14,Symptom_15,Symptom_16,Symptom_17
0,Fungal infection,itching,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Fungal infection,skin_rash,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Fungal infection,itching,nodal_skin_eruptions,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fungal infection,itching,skin_rash,dischromic _patches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Fungal infection,itching,skin_rash,nodal_skin_eruptions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [164]:
# Use to encode the data as given weights
symptoms = df_severity.set_index('Symptom').T.to_dict()
weights = {}
for i in symptoms:
  weights[i] = symptoms[i]['weight']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


In [165]:
df = df.fillna(0)

y = df.Disease
X = df.drop('Disease', axis=1)

for feature in X:
  X[feature] = X[feature].str.strip()
  X[feature] = X[feature].str.replace(' ', '_') # filled in
  X[feature] = X[feature].str.replace('__', '_') # filled in

  X[feature] = X[feature].map(weights)
  X[feature] = X[feature].fillna(0)

# Save
# filename = '/content/sample_data/processed.csv'
# df = X.append(y, ignore_index=True)
# df.to_csv(filename, index=False)

In [166]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=200)

def Knn():
    model = RandomForestClassifier()
    model.fit(X_train, y_train)
    
    y_pred_class = model.predict(X_test)

    print('RESULT')
    print('Accuracy:', metrics.accuracy_score(y_test, y_pred_class))
    # print('Precision:', metrics.precision_score(y_test, y_pred_class))
    # print('AUC Score:', metrics.roc_auc_score(y_test, y_pred_class))
    # print('Cross-validated AUC:', cross_val_score(model, X, y, cv=10, scoring='roc_auc').mean())

    filename = '/content/sample_data/model.sav'
    pickle.dump(model, open(filename, 'wb'))

Knn()

RESULT
Accuracy: 0.9853658536585366


In [134]:
df_severity['Symptom'].unique()

array(['itching', 'skin_rash', 'nodal_skin_eruptions',
       'continuous_sneezing', 'shivering', 'chills', 'joint_pain',
       'stomach_pain', 'acidity', 'ulcers_on_tongue', 'muscle_wasting',
       'vomiting', 'burning_micturition', 'spotting_urination', 'fatigue',
       'weight_gain', 'anxiety', 'cold_hands_and_feets', 'mood_swings',
       'weight_loss', 'restlessness', 'lethargy', 'patches_in_throat',
       'irregular_sugar_level', 'cough', 'high_fever', 'sunken_eyes',
       'breathlessness', 'sweating', 'dehydration', 'indigestion',
       'headache', 'yellowish_skin', 'dark_urine', 'nausea',
       'loss_of_appetite', 'pain_behind_the_eyes', 'back_pain',
       'constipation', 'abdominal_pain', 'diarrhoea', 'mild_fever',
       'yellow_urine', 'yellowing_of_eyes', 'acute_liver_failure',
       'fluid_overload', 'swelling_of_stomach', 'swelled_lymph_nodes',
       'malaise', 'blurred_and_distorted_vision', 'phlegm',
       'throat_irritation', 'redness_of_eyes', 'sinus_pressu

In [151]:
def input_training(row):
  val = []
  for x in X_train.iloc[row]:
    val.append(x)

  return [val]

def input_custom(symptoms):
  val = []
  for index, value in enumerate(symptoms):
    try:
      val.append(weights[value])   # Encode them by the weights
    except:
      val.append(0)

  # Fill remaining symptoms list with zeros
  for i in range(0,17-len(symptoms)):
    val.append(0)

  return [val]

In [146]:
model = pickle.load(open('/content/sample_data/model.sav', 'rb'))   # load model

def custom_predict(symptoms):
  input = input_custom(symptoms)
  print(input)
  output = model.predict(input)

  print("X=%s, Predicted=%s" % (input[0], output[0]))

def training_predict(row):
  input = input_training(row)
  output = model.predict(input)

  print("X=%s, Predicted=%s, Actually=%s" % (input[0], output[0], y_train.iloc[row]))

In [152]:
symptoms = ['itching', 'skin_rash', 'nodal_skin_eruptions', 'dischromic _patches']
custom_predict(symptoms)

# row = 89
# training_predict(row)

[[1, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
X=[1, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], Predicted=Fungal infection


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  "X does not have valid feature names, but"
